## Preparing SV Dataset

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

#np.set_printoptions(formatter={'int':hex})

import os
files = os.listdir('sv_traces/')

### reads an sv_trace file into 2 structures:
### pcs: a list of 1001 16-bit pc values
### mem: a list of 1001 65542-byte memory vectors
def read_trace(sv_file):
    with open(sv_file, 'rb') as f:
        lines = f.read()
        lines = np.frombuffer(lines, dtype=np.uint8)
        lines = lines.reshape(1001,-1)
        pcs = lines[:, 0:2]
        mem = lines[:, :]
        #pcs = [np.uint16(i[1]<<8 | i[0]) for i in pcs]
        return mem, pcs
    
### reads trace data into a dataset of mem-to-pc mappings, where
### mem_current: partial memory trace &
### pcs_current: full pcs trace
def read_dataset(file):
    mem_trace = dict_traces[file][0]
    mem_current = mem_trace[:]
    pcs_trace = dict_traces[file][1]
    pcs_current = pcs_trace[:]
    return mem_current, pcs_current

### dict_traces[file][0]: memory trace of <file>
### dict_traces[file][1]: pc trace of <file>
dict_traces = {}
for file in files[0:20]:
    dict_traces[file] = read_trace(os.path.join('sv_traces/', file))
    
### dict_dataset[file][0]: memory trace of <file>
### dict_dataset[file][1]: pc trace of <file>
dict_dataset = {}
for trace in dict_traces:
    dict_dataset[trace] = read_dataset(trace)

In [2]:
dict_traces

{'167:149:9.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2,   0,   0],
         [  3,   2,   0, ...,   2,   0,   0],
         ...,
         [  3,   2, 126, ...,   2,   0,   0],
         [  5,   2,  20, ...,   2,   0,   0],
         [  7,   2,  20, ...,   2,   0,   0]], dtype=uint8),
  array([[0, 2],
         [1, 2],
         [3, 2],
         ...,
         [3, 2],
         [5, 2],
         [7, 2]], dtype=uint8)),
 '121:187:13.svbinttrc': (array([[ 0,  2,  0, ...,  2,  0,  0],
         [ 1,  2,  0, ...,  2,  0,  0],
         [ 3,  2,  0, ...,  2,  0,  0],
         ...,
         [ 9,  2, 50, ...,  2,  0,  0],
         [10,  2, 50, ...,  2,  0,  0],
         [11,  2, 50, ...,  2,  0,  0]], dtype=uint8),
  array([[ 0,  2],
         [ 1,  2],
         [ 3,  2],
         ...,
         [ 9,  2],
         [10,  2],
         [11,  2]], dtype=uint8)),
 '144:101:13.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2

In [4]:
dict_dataset

{'167:149:9.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2,   0,   0],
         [  3,   2,   0, ...,   2,   0,   0],
         ...,
         [  3,   2, 126, ...,   2,   0,   0],
         [  5,   2,  20, ...,   2,   0,   0],
         [  7,   2,  20, ...,   2,   0,   0]], dtype=uint8),
  array([[0, 2],
         [1, 2],
         [3, 2],
         ...,
         [3, 2],
         [5, 2],
         [7, 2]], dtype=uint8)),
 '121:187:13.svbinttrc': (array([[ 0,  2,  0, ...,  2,  0,  0],
         [ 1,  2,  0, ...,  2,  0,  0],
         [ 3,  2,  0, ...,  2,  0,  0],
         ...,
         [ 9,  2, 50, ...,  2,  0,  0],
         [10,  2, 50, ...,  2,  0,  0],
         [11,  2, 50, ...,  2,  0,  0]], dtype=uint8),
  array([[ 0,  2],
         [ 1,  2],
         [ 3,  2],
         ...,
         [ 9,  2],
         [10,  2],
         [11,  2]], dtype=uint8)),
 '144:101:13.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2

## Defining Neural Net

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
def format_label(l, width=16):
    val = bin(l)[2:]
    N = len(val)
    return [0.] * (width-N) + [float(i) for i in val]

In [5]:
format_label(512)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [6]:
features, labels = dict_dataset[list(dict_dataset.keys())[0]]
for file in list(dict_dataset.keys())[1:10]:
    features = np.append(features, dict_dataset[file][0], axis=0)
    labels = np.append(labels, dict_dataset[file][1], axis=0)

features_test, labels_test = dict_dataset[list(dict_dataset.keys())[10]]
for file in list(dict_dataset.keys())[11:13]:
    features_test = np.append(features_test, dict_dataset[file][0], axis=0)
    labels_test = np.append(labels_test, dict_dataset[file][1], axis=0)

In [9]:
#features, labels = dict_dataset[list(dict_dataset.keys())[0]]
#features_test, labels_test = dict_dataset[list(dict_dataset.keys())[1]]
#features = features - features.mean(axis=0)
# temp
features_temp = features#[9:15, ]
labels_temp = labels#[9:15]
# end temp

In [9]:
print(labels_test[0:16])
print(labels_temp[0:16])


[[ 0  2]
 [ 1  2]
 [ 3  2]
 [ 5  2]
 [ 8  2]
 [ 9  2]
 [10  2]
 [11  2]
 [12  2]
 [13  2]
 [14  2]
 [15  2]
 [16  2]
 [17  2]
 [ 3  2]
 [ 5  2]]
[[ 0  2]
 [ 1  2]
 [ 3  2]
 [ 5  2]
 [ 7  2]
 [ 8  2]
 [ 9  2]
 [10  2]
 [11  2]
 [12  2]
 [13  2]
 [14  2]
 [15  2]
 [16  2]
 [ 3  2]
 [ 5  2]]


In [10]:
labels_temp[2]

array([3, 2], dtype=uint8)

In [11]:
partial_mem_trace = features_temp[: , 0:600]
#partial_mem_trace = features_temp[: , 0:1024]
features_temp = partial_mem_trace # all rows are identical here

partial_mem_trace_test = features_test[: , 0:600]
#partial_mem_trace_test = features_test[: , 0:1024]
features_test = partial_mem_trace_test # all rows are identical here

    
print(partial_mem_trace.shape[0])
print(features_temp.shape[0])
print(partial_mem_trace.shape[1])
print(features_temp.shape[1])

10010
10010
600
600


In [12]:
features_temp[0]

array([  0,   2,   0,   0,   0,   0,  20,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

### Training NN

In [33]:
net = nn.Sequential(
    nn.Linear(features_temp.shape[1], 2**8),
    nn.ReLU(), #rectified linear unit
    nn.Linear(2**8, 2**12),
    nn.ReLU(),
    #nn.Linear(2**8, 2**4),
    #nn.ReLU(),
    nn.Linear(2**12, 2),
    #nn.Sigmoid()
)

#criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()

def train(n_epochs, net, features, labels, criterion, optimizer, freq=1, debug=False):
    for i in range(n_epochs): #number of passes over full dataset
        #step 1: get features and use net to make predictions
        preds = net(torch.from_numpy(features).float())
        
        #step 2: compute loss/error
        labels_torch = torch.tensor(torch.from_numpy(np.array(labels)).float())
        if debug: print('\n------')
        if debug: print(preds)
        #loss = np.sqrt(((np.rint(preds.detach().numpy()) - labels_torch.detach().numpy())**2).mean())
        loss = criterion(preds, labels_torch)
        if i % freq == 0:
            print('epoch:', i, 'loss:', loss)

        #step 3: backprop to update weights
        # compute gradients/derivatives - backprop
        # use gradients to update weights - gradient descent - w = w - 0.1 * deriv. loss w.r.t. w

        optimizer.zero_grad() #set previous buffers to zero
        loss.backward() #backprop
        optimizer.step() #update weights        
        
    return net

optimizer = optim.Adam(net.parameters(), lr=1e-6)
print(features_temp)
#net = train(1000, net, features_temp, labels_temp, criterion, optimizer, debug=False)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
optimizer = optim.Adam(net.parameters(), lr=1e-5)

In [35]:
net = train(5000, net, features_temp, labels_temp, criterion, optimizer, debug=False)

/tmp/ipykernel_109417/2880589121.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_torch = torch.tensor(torch.from_numpy(np.array(labels)).float())


epoch: 0 loss: tensor(95.9756, grad_fn=<MseLossBackward0>)
epoch: 1 loss: tensor(89.6505, grad_fn=<MseLossBackward0>)
epoch: 2 loss: tensor(83.5865, grad_fn=<MseLossBackward0>)
epoch: 3 loss: tensor(77.7861, grad_fn=<MseLossBackward0>)
epoch: 4 loss: tensor(72.2498, grad_fn=<MseLossBackward0>)
epoch: 5 loss: tensor(66.9761, grad_fn=<MseLossBackward0>)
epoch: 6 loss: tensor(61.9637, grad_fn=<MseLossBackward0>)
epoch: 7 loss: tensor(57.2113, grad_fn=<MseLossBackward0>)
epoch: 8 loss: tensor(52.7171, grad_fn=<MseLossBackward0>)
epoch: 9 loss: tensor(48.4797, grad_fn=<MseLossBackward0>)
epoch: 10 loss: tensor(44.4979, grad_fn=<MseLossBackward0>)
epoch: 11 loss: tensor(40.7705, grad_fn=<MseLossBackward0>)
epoch: 12 loss: tensor(37.2952, grad_fn=<MseLossBackward0>)
epoch: 13 loss: tensor(34.0690, grad_fn=<MseLossBackward0>)
epoch: 14 loss: tensor(31.0885, grad_fn=<MseLossBackward0>)
epoch: 15 loss: tensor(28.3491, grad_fn=<MseLossBackward0>)
epoch: 16 loss: tensor(25.8450, grad_fn=<MseLossBa

epoch: 138 loss: tensor(8.7595, grad_fn=<MseLossBackward0>)
epoch: 139 loss: tensor(8.7577, grad_fn=<MseLossBackward0>)
epoch: 140 loss: tensor(8.7558, grad_fn=<MseLossBackward0>)
epoch: 141 loss: tensor(8.7539, grad_fn=<MseLossBackward0>)
epoch: 142 loss: tensor(8.7521, grad_fn=<MseLossBackward0>)
epoch: 143 loss: tensor(8.7502, grad_fn=<MseLossBackward0>)
epoch: 144 loss: tensor(8.7484, grad_fn=<MseLossBackward0>)
epoch: 145 loss: tensor(8.7466, grad_fn=<MseLossBackward0>)
epoch: 146 loss: tensor(8.7448, grad_fn=<MseLossBackward0>)
epoch: 147 loss: tensor(8.7430, grad_fn=<MseLossBackward0>)
epoch: 148 loss: tensor(8.7412, grad_fn=<MseLossBackward0>)
epoch: 149 loss: tensor(8.7394, grad_fn=<MseLossBackward0>)
epoch: 150 loss: tensor(8.7377, grad_fn=<MseLossBackward0>)
epoch: 151 loss: tensor(8.7359, grad_fn=<MseLossBackward0>)
epoch: 152 loss: tensor(8.7342, grad_fn=<MseLossBackward0>)
epoch: 153 loss: tensor(8.7325, grad_fn=<MseLossBackward0>)
epoch: 154 loss: tensor(8.7308, grad_fn=

epoch: 275 loss: tensor(8.5471, grad_fn=<MseLossBackward0>)
epoch: 276 loss: tensor(8.5456, grad_fn=<MseLossBackward0>)
epoch: 277 loss: tensor(8.5440, grad_fn=<MseLossBackward0>)
epoch: 278 loss: tensor(8.5425, grad_fn=<MseLossBackward0>)
epoch: 279 loss: tensor(8.5409, grad_fn=<MseLossBackward0>)
epoch: 280 loss: tensor(8.5394, grad_fn=<MseLossBackward0>)
epoch: 281 loss: tensor(8.5378, grad_fn=<MseLossBackward0>)
epoch: 282 loss: tensor(8.5362, grad_fn=<MseLossBackward0>)
epoch: 283 loss: tensor(8.5347, grad_fn=<MseLossBackward0>)
epoch: 284 loss: tensor(8.5331, grad_fn=<MseLossBackward0>)
epoch: 285 loss: tensor(8.5316, grad_fn=<MseLossBackward0>)
epoch: 286 loss: tensor(8.5300, grad_fn=<MseLossBackward0>)
epoch: 287 loss: tensor(8.5284, grad_fn=<MseLossBackward0>)
epoch: 288 loss: tensor(8.5269, grad_fn=<MseLossBackward0>)
epoch: 289 loss: tensor(8.5253, grad_fn=<MseLossBackward0>)
epoch: 290 loss: tensor(8.5238, grad_fn=<MseLossBackward0>)
epoch: 291 loss: tensor(8.5222, grad_fn=

epoch: 412 loss: tensor(8.3316, grad_fn=<MseLossBackward0>)
epoch: 413 loss: tensor(8.3300, grad_fn=<MseLossBackward0>)
epoch: 414 loss: tensor(8.3284, grad_fn=<MseLossBackward0>)
epoch: 415 loss: tensor(8.3269, grad_fn=<MseLossBackward0>)
epoch: 416 loss: tensor(8.3253, grad_fn=<MseLossBackward0>)
epoch: 417 loss: tensor(8.3237, grad_fn=<MseLossBackward0>)
epoch: 418 loss: tensor(8.3222, grad_fn=<MseLossBackward0>)
epoch: 419 loss: tensor(8.3206, grad_fn=<MseLossBackward0>)
epoch: 420 loss: tensor(8.3190, grad_fn=<MseLossBackward0>)
epoch: 421 loss: tensor(8.3175, grad_fn=<MseLossBackward0>)
epoch: 422 loss: tensor(8.3159, grad_fn=<MseLossBackward0>)
epoch: 423 loss: tensor(8.3143, grad_fn=<MseLossBackward0>)
epoch: 424 loss: tensor(8.3127, grad_fn=<MseLossBackward0>)
epoch: 425 loss: tensor(8.3111, grad_fn=<MseLossBackward0>)
epoch: 426 loss: tensor(8.3095, grad_fn=<MseLossBackward0>)
epoch: 427 loss: tensor(8.3080, grad_fn=<MseLossBackward0>)
epoch: 428 loss: tensor(8.3064, grad_fn=

...

epoch: 4335 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4336 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4337 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4338 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4339 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4340 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4341 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4342 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4343 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4344 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4345 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4346 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4347 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4348 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4349 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4350 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4351 loss: tensor

epoch: 4470 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4471 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4472 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4473 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4474 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4475 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4476 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4477 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4478 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4479 loss: tensor(0.0021, grad_fn=<MseLossBackward0>)
epoch: 4480 loss: tensor(0.0021, grad_fn=<MseLossBackward0>)
epoch: 4481 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4482 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4483 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4484 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4485 loss: tensor(0.0021, grad_fn=<MseLossBackward0>)
epoch: 4486 loss: tensor

epoch: 4605 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4606 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4607 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4608 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4609 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4610 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4611 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4612 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4613 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4614 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4615 loss: tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch: 4616 loss: tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch: 4617 loss: tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch: 4618 loss: tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch: 4619 loss: tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch: 4620 loss: tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch: 4621 loss: tensor

epoch: 4740 loss: tensor(0.0027, grad_fn=<MseLossBackward0>)
epoch: 4741 loss: tensor(0.0033, grad_fn=<MseLossBackward0>)
epoch: 4742 loss: tensor(0.0033, grad_fn=<MseLossBackward0>)
epoch: 4743 loss: tensor(0.0026, grad_fn=<MseLossBackward0>)
epoch: 4744 loss: tensor(0.0019, grad_fn=<MseLossBackward0>)
epoch: 4745 loss: tensor(0.0016, grad_fn=<MseLossBackward0>)
epoch: 4746 loss: tensor(0.0018, grad_fn=<MseLossBackward0>)
epoch: 4747 loss: tensor(0.0022, grad_fn=<MseLossBackward0>)
epoch: 4748 loss: tensor(0.0024, grad_fn=<MseLossBackward0>)
epoch: 4749 loss: tensor(0.0024, grad_fn=<MseLossBackward0>)
epoch: 4750 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4751 loss: tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch: 4752 loss: tensor(0.0016, grad_fn=<MseLossBackward0>)
epoch: 4753 loss: tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch: 4754 loss: tensor(0.0019, grad_fn=<MseLossBackward0>)
epoch: 4755 loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
epoch: 4756 loss: tensor

epoch: 4875 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4876 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4877 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4878 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4879 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4880 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4881 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4882 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4883 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4884 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4885 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4886 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4887 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4888 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4889 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4890 loss: tensor(0.0014, grad_fn=<MseLossBackward0>)
epoch: 4891 loss: tensor

In [26]:
#net = net.to(device)
N_epochs = 10000

labels_temp = np.array(labels_temp)
#train_data_torch = torch.from_numpy(features_temp).float()
#train_target_torch = torch.from_numpy(labels_temp).float()
train_target_torch = torch.tensor(torch.from_numpy(np.array(labels_temp)).float())

import sys
np.set_printoptions(threshold=sys.maxsize)

def validate(net, test_data, test_target, i):
    
    test_data_torch = torch.from_numpy(features_test).float()
    #test_target_torch = torch.from_numpy(labels_test).float()
    test_target_torch = torch.tensor(torch.from_numpy(np.array(test_target)).float())

    test_pred = np.rint(net(test_data_torch).detach().numpy())
    if i % 500 == 0:
        print(np.concatenate((test_pred, test_target_torch.detach().numpy()), axis=1))
    #test_accuracy = (test_target_torch.detach().numpy() == test_pred).float().mean().detach().numpy()
    test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()

    #test_rmse = ((test_pred-test_target_torch)**2).mean().sqrt().detach().numpy()
    test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse

for i in range(1):
    #pred = net(train_data_torch)
    #loss = criterion(pred, train_target_torch)
    
    #optimizer.zero_grad()
    #loss.backward()
    #optimizer.step()
    
    if i % 100 == 0:
        #train_accuracy, train_rmse = validate(net, features_temp, labels_temp)
        test_accuracy, test_rmse = validate(net, features_test, labels_test, i)
        #print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')         
        print(f'Epoch = {i} test_acc = {test_accuracy:.3f}  test_rmse = {test_rmse:.3f}') 

[[ 1.  4.  0.  2.]
 [ 2.  4.  1.  2.]
 [ 4.  4.  3.  2.]
 [ 5.  3.  5.  2.]
 [ 8.  2.  8.  2.]
 [ 9.  2.  9.  2.]
 [10.  2. 10.  2.]
 [11.  2. 11.  2.]
 [11.  2. 12.  2.]
 [12.  2. 13.  2.]
 [13.  2. 14.  2.]
 [14.  2. 15.  2.]
 [15.  2. 16.  2.]
 [16.  2. 17.  2.]
 [ 3.  2.  3.  2.]
 [ 5.  3.  5.  2.]
 [ 8.  3.  8.  2.]
 [ 9.  3.  9.  2.]
 [10.  3. 10.  2.]
 [10.  3. 11.  2.]
 [11.  3. 12.  2.]
 [12.  3. 13.  2.]
 [13.  3. 14.  2.]
 [14.  3. 15.  2.]
 [15.  3. 16.  2.]
 [16.  3. 17.  2.]
 [ 3.  3.  3.  2.]
 [ 5.  3.  5.  2.]
 [ 8.  3.  8.  2.]
 [ 9.  3.  9.  2.]
 [10.  3. 10.  2.]
 [10.  3. 11.  2.]
 [11.  3. 12.  2.]
 [12.  3. 13.  2.]
 [13.  3. 14.  2.]
 [14.  3. 15.  2.]
 [15.  3. 16.  2.]
 [16.  3. 17.  2.]
 [ 3.  3.  3.  2.]
 [ 5.  3.  5.  2.]
 [ 8.  3.  8.  2.]
 [ 9.  3.  9.  2.]
 [10.  3. 10.  2.]
 [11.  3. 11.  2.]
 [11.  3. 12.  2.]
 [12.  3. 13.  2.]
 [13.  3. 14.  2.]
 [14.  3. 15.  2.]
 [15.  3. 16.  2.]
 [16.  3. 17.  2.]
 [ 3.  3.  3.  2.]
 [ 5.  3.  5.  2.]
 [ 8.  3.  8

/tmp/ipykernel_109417/3030192740.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_target_torch = torch.tensor(torch.from_numpy(np.array(labels_temp)).float())
/tmp/ipykernel_109417/3030192740.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_target_torch = torch.tensor(torch.from_numpy(np.array(test_target)).float())


In [23]:
preds = net(torch.from_numpy(features_test[3]).float()).detach().numpy()
print(np.rint(preds))

[5. 3.]


In [67]:
for p in preds:
    print(p)

tensor(4.9992, grad_fn=<UnbindBackward0>)
tensor(2.0004, grad_fn=<UnbindBackward0>)


In [60]:
labels_test[3]

array([5, 2], dtype=uint8)